In [ ]:
import numpy as np
import time
from numpy import ones,vstack
from numpy.linalg import lstsq

import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from random import random, randint, uniform

import os
import shutil

from statistics import median
import pickle

In [ ]:
def generate_decoding_domain(observations, radius=10, time_precision=0.05):
    
    pddl_str = "(define (domain flight)\n"
    pddl_str += "\n"
    
    # Predicates
    pddl_str += "(:predicates\n"
    pddl_str += "\t;HS locations\n"
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t(adjusting-left)\n"
    pddl_str += "\t(adjusting-right)\n"
    pddl_str += "\n"    
    pddl_str += "\t;monitor locations\n"
    for i in range(len(observations)+1):
        pddl_str += "\t(monitor_{})\n".format(i)
    pddl_str += ")\n"
    pddl_str += "\n"
    
    # Functions
    pddl_str += "(:functions\n"
    pddl_str += "\t;HS variables\n"
    pddl_str += "\t(x1) ; coordinate x\n"
    pddl_str += "\t(x2) ; coordinate y\n"
    pddl_str += "\t(theta) ; orientation\n"
    pddl_str += "\t(v) ; linear speed\n"
    pddl_str += "\t(w) ; angular speed\n"
    pddl_str += "\n"
    
   
    pddl_str += "\t;monitor variables\n"
    pddl_str += "\t(running_time)  ;; time\n"
    pddl_str += "\n"
    
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-flying-straight\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t 0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t -0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    
    # HS Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Invariants
    
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Invariants\n"
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
             
    
    # Monitor Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    
    for i in range(len(observations)+1):
        pddl_str += "(:process flow-monitor_{}\n".format(i)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(increase (running_time) (* #t 1.0))\n"
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    # Monitor Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
        
    for i in range(len(observations)):
        obs = observations[i]
        
        pddl_str += "(:action validate_{}\n".format(i+1)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t\t(<= {} running_time) ; t_{} - 0.05\n".format(obs[2] - time_precision, i+1)
        pddl_str += "\t\t(> {} running_time) ; t_{} + 0.05\n".format(obs[2] + time_precision, i+1)
        pddl_str += "\t\t(< (+ (^ (- x1 {}) 2) (^ (- x2 {}) 2)) (^ {} 2) ) ; \phi_{}\n".format(round(obs[0],3), round(obs[1],3), radius, i+1)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(not (monitor_{}))\n".format(i)
        pddl_str += "\t\t(monitor_{})\n".format(i+1)
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
       
        
    pddl_str += ")"
    
    
    return pddl_str
    

In [ ]:
def generate_decoding_problem(observations, theta, v):
    
    pddl_str = "(define (problem problem_name) (:domain platoon)\n"
    pddl_str += "\n"

    # Initial state
    pddl_str += "(:init\n"
    pddl_str += "\t; HS initial state\n"
    pddl_str += "\t(= (x1) 0)\n"
    pddl_str += "\t(= (x2) 0)\n"
    pddl_str += "\t(= (theta) {})\n".format(theta)
    pddl_str += "\t(= (v) {})\n".format(v)
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t; monitor initial state\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\t(monitor_0)\n"
    pddl_str += "\n"
    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(monitor_{})\n".format(len(observations))
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str

In [ ]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/flight/ENHSP"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)

with open("flight.pkl", "rb") as f:
    instances = pickle.load(f)

for pnum in range(num_instances):
    for h in horizons:
        dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        
        
        domain_str = generate_decoding_domain(instances[pnum]["observations"][:h])
        with open(dfolder + '/' + dname, 'w') as f:
            f.write(domain_str)
            
        problem_str = generate_decoding_problem(instances[pnum]["observations"][:h], instances[pnum]["init_theta"], instances[pnum]["init_v"])
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(problem_str)

In [ ]:
#horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#dfolder = "benchmarks/flight/HSE_Pe"

times = {}
instances = list(range(num_instances))
for h in horizons:
    h_times = []
    working_instances = []
    print(instances)
    for pnum in instances:  
        
        try:
            dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
            pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)

            cmd = "timeout 300 enhsp_exp -o {dfolder}/{dname} -f {dfolder}/{pname} -dp 0.01 -dh 0.01 -de 0.01 -dv 0.01 -s WAStar -h hmrp -pt -sp {dfolder}/plan_{pnum}_{h}".format(dfolder=dfolder, dname=dname, pname=pname, pnum=pnum, h=h)

            t = time.time()
            os.system(cmd)
            elapsed = time.time() - t

            shutil.move(dfolder + "/{pname}_search_WAStar_h_hmrp_break_ties_arbitrary.npt".format(pname=pname), dfolder + "/solution_{pnum}_{h}.npt".format(pnum=pnum, h=h))
            h_times += [elapsed]
            working_instances += [pnum]
        except:
            print("instance {} failed".format(pnum))
        
    times[h] = h_times
    print(h, h_times)
    
    instances = working_instances
    
with open("{}/enhsp".format(dfolder), "w") as f:
    for k,h_times in times.items():
        f.write(" ".join(map(str, h_times)) + "\n")
